In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import types
import pymysql
pymysql.install_as_MySQLdb()

import os
import sys
sys.path
sys.path.append('../')
sys.path.append('spotify-chart')
import connections
import charts

## Top 200 Daily

In [3]:
daily_charts = charts.get_charts('2017-01-01', '2019-02-28', freq = 'daily', region = 'us')

 19%|███████████████                                                                 | 149/789 [03:54<16:58,  1.59s/it]

 19%|███████████████▏                                                                | 150/789 [03:55<13:23,  1.26s/it]

 19%|███████████████▍                                                                | 152/789 [03:57<12:13,  1.15s/it]

100%|████████████████████████████████████████████████████████████████████████████████| 789/789 [21:24<00:00,  1.61s/it]


In [9]:
top_200_daily = pd.DataFrame(daily_charts)
top_200_daily.rename(columns = {'Unnamed: 0':'Position', 
                                'Unnamed: 1': 'Track Name', 
                                'Unnamed: 2':'Artist', 
                                'Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.':'Streams', 
                                'Unnamed: 4':'URL'}, inplace = True)
top_200_daily.head()

,Position,Track Name,Artist,Streams,URL,date
0,Position,Track Name,Artist,Streams,URL,2017-01-01
1,1,Bad and Boujee (feat. Lil Uzi Vert),Migos,1371493,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,2017-01-01
2,2,Fake Love,Drake,1180074,https://open.spotify.com/track/343YBumqHu19cGo...,2017-01-01
3,3,Starboy,The Weeknd,1064351,https://open.spotify.com/track/5aAx2yezTd8zXrk...,2017-01-01
4,4,Closer,The Chainsmokers,1010492,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,2017-01-01


In [4]:
top_200_daily.drop(columns = 'Unnamed: 0', inplace = True)
# top_200_weekly.rename(columns = {'Unnamed: 0':'index'}, inplace = True)
top_200_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

top_200_daily['ID'] = top_200_daily['URL'].str.split('/', expand = True)[4]

# top 200 db export
top_200_db = top_200_daily[['Position', 'date', 'ID']]
top_200_db.head()

KeyError: 'URL'

## Viral 50 Daily

In [ ]:
viral_charts = charts.get_charts('2017-01-01', '2019-02-28', chart = 'viral', freq = 'daily', region = 'us')

  3%|██▎                                                                              | 22/789 [00:38<21:49,  1.71s/it]

In [ ]:
viral_50_daily = pd.DataFrame(viral_charts)

viral_50_daily.drop(columns = 'Unnamed: 0', inplace = True)
# top_200_weekly.rename(columns = {'Unnamed: 0':'index'}, inplace = True)
viral_50_daily.rename(columns = {'Track Name':'Track'}, inplace = True)

viral_50_daily['ID'] = viral_50_daily['URL'].str.split('/', expand = True)[4]

# top 200 db export
viral_50_db = viral_50_daily[['Position', 'date', 'ID']]
viral_50_db.head()

## Unique Songs

In [3]:
unique_songs = viral_50_daily.drop_duplicates('ID')[['ID', 'Track', 'Artist']]
unique_songs.head()

NameError: name 'viral_50_daily' is not defined

## Features

# Export Song Tables to SQL

In [29]:
connection_string = (f"root:{password}@localhost")

engine = create_engine(f"mysql://{connection_string}")
engine.execute("DROP DATABASE IF EXISTS spot_db")
engine.execute("CREATE DATABASE spot_db")
engine.execute("ALTER DATABASE spot_db CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci")
# engine.execute("ALTER TABLE indeed_resumes CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
engine = create_engine(f"mysql://{connection_string}/spot_db?charset=utf8mb4")

In [30]:
# Top 200 Daily
engine.execute("USE spot_db")
top_200_db.to_sql(
    name = 'top_200_daily', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `top_200_daily` ADD PRIMARY KEY (`Index`);')

In [31]:
# Viral 50 Daily
engine.execute("USE spot_db")
viral_50_db.to_sql(
    name = 'viral_50_daily', con = engine,
    if_exists = 'replace')
with engine.connect() as con:
    con.execute('ALTER TABLE `viral_50_daily` ADD PRIMARY KEY (`Index`);')

In [60]:
# Unique Songs
engine.execute("USE spot_db")
unique_songs.to_sql(
    name = 'songs', con = engine,
    if_exists = 'replace', 
    dtype={'ID': types.VARCHAR(255)})
with engine.connect() as con:
    con.execute('ALTER TABLE `songs` ADD PRIMARY KEY (`Id`);')